## Importing Libraries

In [1]:
import numpy as np 
import pandas as pd
import glob 
import os

## Reading multiple station Data (+Extracting Meta Data)

In [2]:
def get_csv_files():
    current_dir = os.getcwd()
    parent_dir = os.path.dirname(current_dir)
    raw_data_dir = os.path.join(parent_dir, "Raw Station Datasets")
    csv_files = glob.glob(os.path.join(raw_data_dir, '*.csv'))
    filtered_files = sorted([file for file in csv_files if os.path.basename(file)[:3].isdigit()],
                            key=lambda x: int(os.path.basename(x)[:3]))
    return filtered_files

def load_csv_files(file_list):
    dfs = []
    for file in file_list:
        try:
            # Extract Metadata + Header data
            with open(file, 'r') as f:
                meta_lines = [f.readline().strip() for _ in range(12)]

            # Extract only metadata from meta_lines
            metadata = {}
            for line in meta_lines[4:10]:  # Lines 5-10
                key, value = line.split(': ')
                metadata[key.strip('# ')] = value

            # Combine the Parameter + it's Units (lines 11 and 12)
            headers = meta_lines[10].split(',')  # Line 11
            units = meta_lines[11].split(',')  # Line 12

            new_headers = []
            for header, unit in zip(headers, units):
                if unit.strip():
                    new_headers.append(f'{header.strip()}({unit.strip()})')
                else:
                    new_headers.append(header.strip())  # Account for no units, first two rows (Station_ID and Date_time)

            # Add data starting from Line 13 for every csv file
            df = pd.read_csv(file, skiprows=12, names=new_headers)
            
            # Create new columns hosting meta data (Will be converted to Categorical Variable later)
            for key, value in metadata.items():
                df[key] = value

            # Appending each DataFrame to the list
            dfs.append(df)
            print(f"Successfully loaded: {os.path.basename(file)}")
        except Exception as e:
            print(f"Error loading {os.path.basename(file)}: {e}")
    return dfs

# Loading Step
csv_files = get_csv_files()
dfs = load_csv_files(csv_files)
print(f"\nNumber of DataFrames loaded: {len(dfs)}")

Successfully loaded: 001HI.2024-10-07.csv
Successfully loaded: 002HI.2024-10-07.csv
Successfully loaded: 003HI.2024-10-13.csv
Successfully loaded: 004HI.2024-10-13.csv
Successfully loaded: 005HI.2024-10-13.csv
Successfully loaded: 006HI.2024-10-13.csv
Successfully loaded: 007HI.2024-10-13.csv
Successfully loaded: 008HI.2024-10-13.csv
Successfully loaded: 009HI.2024-10-13.csv
Successfully loaded: 010HI.2024-10-13.csv
Successfully loaded: 011HI.2024-10-13.csv


/var/folders/8v/j9k9_w615x5dxs8km9fp3nmc0000gn/T/ipykernel_6129/834843614.py:36: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, skiprows=12, names=new_headers)


Successfully loaded: 012HI.2024-10-13.csv
Successfully loaded: 013HI.2024-10-13.csv
Successfully loaded: 014HI.2024-10-13.csv
Successfully loaded: 015HI.2024-10-13.csv
Successfully loaded: 016HI.2024-10-13.csv
Successfully loaded: 017HI.2024-10-07.csv
Successfully loaded: 018HI.2024-10-20.csv
Successfully loaded: 019HI.2024-10-20.csv
Successfully loaded: 020HI.2024-10-20.csv
Successfully loaded: 021HI.2024-10-20.csv
Successfully loaded: 022HI.2024-10-20.csv
Successfully loaded: 023HI.2024-05-16.csv


/var/folders/8v/j9k9_w615x5dxs8km9fp3nmc0000gn/T/ipykernel_6129/834843614.py:36: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, skiprows=12, names=new_headers)


Successfully loaded: 024HI.2024-10-20.csv
Successfully loaded: 025HI.2024-10-20.csv
Successfully loaded: 026HI.2024-10-20.csv
Successfully loaded: 027HI.2024-10-20.csv
Successfully loaded: 028HI.2024-09-03.csv
Successfully loaded: 029HI.2024-10-20.csv
Successfully loaded: 030HI.2024-10-20.csv
Successfully loaded: 031HI.2024-10-20.csv
Successfully loaded: 032HI.2024-11-10.csv
Successfully loaded: 033HI.2024-11-10.csv
Successfully loaded: 034HI.2024-11-10.csv
Successfully loaded: 035HI.2024-11-10.csv
Successfully loaded: 036HI.2024-11-10.csv
Successfully loaded: 037HI.2024-11-10.csv
Successfully loaded: 038HI.2024-11-10.csv
Successfully loaded: 039HI.2024-11-10.csv
Successfully loaded: 040HI.2024-11-10.csv
Successfully loaded: 041HI.2024-11-10.csv
Successfully loaded: 042HI.2024-11-10.csv
Successfully loaded: 043HI.2024-06-16.csv
Successfully loaded: 044HI.2024-06-16.csv
Successfully loaded: 045HI.2024-06-16.csv
Successfully loaded: 046HI.2024-11-18.csv
Successfully loaded: 047HI.2024-11

In [3]:
dfs[10] #Accessing, only the first Station_ID file! 

,Station_ID,Date_Time,pressure_set_1(Pascals),air_temp_set_1(Celsius),relative_humidity_set_1(%),wind_speed_set_1(m/s),wind_direction_set_1(Degrees),wind_gust_set_1(m/s),solar_radiation_set_1(W/m**2),soil_temp_set_1(Celsius),...,dew_point_temperature_set_1d(Celsius),altimeter_set_1d(Pascals),sea_level_pressure_set_1d(Pascals),wet_bulb_temp_set_1d(Celsius),STATION,STATION NAME,LATITUDE,LONGITUDE,ELEVATION [ft],STATE
0,011HI,2022-04-25T00:00:00Z,NaN,23.56,76.89,0.40,178.2,NaN,330.61,24.46,...,19.26,NaN,NaN,NaN,011HI,Lyon,21.33300,-157.80250,495.0,HI
1,011HI,2022-04-25T00:05:00Z,NaN,23.60,76.87,0.44,235.4,NaN,577.30,24.51,...,19.29,NaN,NaN,NaN,011HI,Lyon,21.33300,-157.80250,495.0,HI
2,011HI,2022-04-25T00:10:00Z,NaN,23.86,76.76,0.67,167.9,NaN,259.37,24.55,...,19.52,NaN,NaN,NaN,011HI,Lyon,21.33300,-157.80250,495.0,HI
3,011HI,2022-04-25T00:15:00Z,NaN,23.48,76.34,0.77,178.3,NaN,277.75,24.63,...,19.06,NaN,NaN,NaN,011HI,Lyon,21.33300,-157.80250,495.0,HI
4,011HI,2022-04-25T00:20:00Z,NaN,23.47,75.77,0.55,222.3,NaN,405.45,24.70,...,18.93,NaN,NaN,NaN,011HI,Lyon,21.33300,-157.80250,495.0,HI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247475,011HI,2024-10-13T11:40:00Z,99840.0,22.90,87.50,0.27,202.3,1.00,0.00,24.56,...,20.70,101646.13,101590.34,21.40,011HI,Lyon,21.33300,-157.80250,495.0,HI
247476,011HI,2024-10-13T11:45:00Z,99840.0,22.70,89.90,0.24,209.1,0.89,0.00,24.57,...,20.95,101646.13,101591.53,21.51,011HI,Lyon,21.33300,-157.80250,495.0,HI
247477,011HI,2024-10-13T11:50:00Z,99830.0,22.56,90.40,0.26,241.9,1.28,0.00,24.58,...,20.90,101635.95,101582.18,21.44,011HI,Lyon,21.33300,-157.80250,495.0,HI
247478,011HI,2024-10-13T11:55:00Z,99830.0,22.41,91.40,1.00,155.9,2.00,0.00,24.58,...,20.93,101635.95,101583.07,21.42,011HI,Lyon,21.33300,-157.80250,495.0,HI


In [4]:
dfs[10].to_csv("011HI.csv", index=False)

## Concatenate all Station Data into a single Dataframe

In [4]:
# Concatenate all DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

# Converting the columns which merely get replicated to categorical variable to save space!
categorical_columns = ['Station_ID', 'STATION', 'STATION NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION [ft]', 'STATE']

for col in categorical_columns:
    if col in combined_df.columns:
        combined_df[col] = combined_df[col].astype('category')

KeyboardInterrupt: 

In [5]:
combined_df[combined_df['Station_ID'] == '017HI'] #Accessing specific Station's Data

,Station_ID,Date_Time,pressure_set_1(Pascals),air_temp_set_1(Celsius),relative_humidity_set_1(%),wind_speed_set_1(m/s),wind_direction_set_1(Degrees),wind_gust_set_1(m/s),solar_radiation_set_1(W/m**2),soil_temp_set_1(Celsius),...,STATION,STATION NAME,LATITUDE,LONGITUDE,ELEVATION [ft],STATE,soil_temp_set_2(Celsius),soil_temp_set_3(Celsius),soil_temp_set_4(Celsius),soil_moisture_set_3(%)
508043,017HI,2022-12-05T00:00:00Z,NaN,21.85,72.28,0.81,116.1,NaN,232.55,20.53,...,017HI,Piiholo,20.8415,-156.2948,2090.0,HI,19.97,20.60,NaN,43.9
508044,017HI,2022-12-05T00:05:00Z,NaN,21.62,72.28,2.40,90.8,NaN,381.14,20.56,...,017HI,Piiholo,20.8415,-156.2948,2090.0,HI,19.99,20.58,NaN,43.9
508045,017HI,2022-12-05T00:10:00Z,NaN,21.75,73.15,2.20,99.3,NaN,467.97,20.60,...,017HI,Piiholo,20.8415,-156.2948,2090.0,HI,20.03,20.61,NaN,43.9
508046,017HI,2022-12-05T00:15:00Z,NaN,22.06,71.56,3.11,91.1,NaN,755.71,20.63,...,017HI,Piiholo,20.8415,-156.2948,2090.0,HI,20.06,20.59,NaN,43.9
508047,017HI,2022-12-05T00:20:00Z,NaN,21.88,70.89,1.76,89.5,NaN,271.18,20.68,...,017HI,Piiholo,20.8415,-156.2948,2090.0,HI,20.05,20.58,NaN,43.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700826,017HI,2024-10-07T11:40:00Z,94450.0,16.96,71.69,1.40,95.7,2.17,0.00,NaN,...,017HI,Piiholo,20.8415,-156.2948,2090.0,HI,20.96,21.37,20.58,37.7
700827,017HI,2024-10-07T11:45:00Z,94450.0,16.99,72.43,1.32,151.1,2.28,0.00,NaN,...,017HI,Piiholo,20.8415,-156.2948,2090.0,HI,20.99,21.38,20.57,37.7
700828,017HI,2024-10-07T11:50:00Z,94450.0,16.80,73.51,1.29,151.7,1.89,0.00,NaN,...,017HI,Piiholo,20.8415,-156.2948,2090.0,HI,20.95,21.37,20.56,37.7
700829,017HI,2024-10-07T11:55:00Z,94440.0,16.90,74.96,1.12,156.5,2.28,0.00,NaN,...,017HI,Piiholo,20.8415,-156.2948,2090.0,HI,20.92,21.39,20.58,37.7


## Saving the Reformatted + Cleaned Dataframe as .CSV

In [46]:
csv_filename = "combined_data.csv"
combined_df.to_csv(csv_filename, index=False)